порядок записи ключевых слов выглядит так:

SELECT-
FROM-
WHERE-
GROUP BY-
ORDER BY-
LIMIT

В то же время порядок выполнения операторов в запросе следующий:

Сначала выполняется оператор FROM — происходит выбор нужной таблицы.

Далее WHERE — отфильтровываются строки, соответствующие условию.

Потом GROUP BY — строки объединяются в группы и производится агрегация.

Затем SELECT — отбираются указанные столбцы.

Потом ORDER BY — производится сортировка результирующей таблицы.

И в самом конце LIMIT — ограничивается количество выводимых записей.


**___________________________________________________________________**

К группам, образовавшимся в результате GROUP BY, можно применять сразу несколько агрегирующих функций (в том числе к разным колонкам):

SELECT column_1, SUM(column_2) AS sum, AVG(column_3) AS avg \
FROM table \
GROUP BY column_1

SELECT column_1, COUNT(column_1) AS count\
FROM table\
GROUP BY column_1


**___________________________________________________________________**

После группировки к результатам агрегирующих функций можно применять другие функции. Также над ними можно сразу проводить арифметические операции:

SELECT column_1, ROUND(SUM(column_2), 2) AS rounded_sum\
FROM table\
GROUP BY column_1


SELECT column_1, SUM(column_2)::DECIMAL / SUM(column_3) * 100 AS share\
FROM table\
GROUP BY column_1


**___________________________________________________________________**

Агрегацию не обязательно проводить по уже имеющимся колонкам — можно сначала применить некоторую функцию в колонке из исходной таблицы, а затем сразу провести агрегацию по новой колонке:

SELECT column_1, SUM(some_function(column_2)) AS sum_over_function\
FROM table\
GROUP BY column_1


**___________________________________________________________________**

Группировку можно делать сразу по новым полям, посчитанным в SELECT. При этом допускается использование в GROUP BY алиаса колонки, указанного в SELECT.

Например, если нам необходимо сгруппировать данные по дням, выделив дату из какой-то колонки, и посчитать сумму некоторых значений в каждый из дней, то можно использовать следующие запросы, которые дадут одинаковый результат:

SELECT DATE(column_1) AS date, SUM(column_2) AS sum\
FROM table\
GROUP BY DATE(column_1)


SELECT DATE(column_1) AS date, SUM(column_2) AS sum\
FROM table\
GROUP BY date


**___________________________________________________________________**

 делать агрегацию после группировки не обязательно. Если не указать агрегирующую функцию, то запрос вернёт уникальные значения в столбце (или уникальные комбинации значений в столбцах), т.е. тот же результат, что и оператор DISTINCT. Можете самостоятельно запустить следующие запросы в Redash и убедиться:

SELECT user_id\
FROM user_actions\
GROUP BY user_id


SELECT DISTINCT user_id\
FROM user_actions


**___________________________________________________________________**

при использовании группировки колонки, указанные в SELECT, должны находиться и в GROUP BY, если они не используются в агрегирующих функциях.

Это обязательное условие, и если оно не будет выполнено, то база данных вернёт ошибку.

SELECT column_1, column_2, SUM(column_3) AS sum\
FROM table\
GROUP BY column_1, column_2

Обратите внимание, что в обратную сторону указанное выше правило не работает: если мы что-то указали в GROUP BY, то это совсем не обязательно указывать в SELECT. Иными словами, можно не выводить наименования колонок, по которым осуществляется группировка.


**___________________________________________________________________**

И наконец, группировка всегда выполняется после фильтрации, т.е. сначала выполняются инструкции в WHERE и только потом данные группируются с помощью GROUP BY:

SELECT column_1, SUM(column_2) AS sum\
FROM table\
WHERE column_3 > 0\
GROUP BY column_1

**#1**

Задание:\
С помощью оператора GROUP BY посчитайте количество курьеров мужского и женского пола в таблице couriers.\
Новую колонку с числом курьеров назовите couriers_count.\
Результат отсортируйте по этой колонке по возрастанию.\
Поля в результирующей таблице: sex, couriers_count

SELECT sex,\
       count(sex) as couriers_count\
FROM   couriers\
GROUP BY sex\
ORDER BY couriers_count


**#2**

Задание:\
Посчитайте количество созданных и отменённых заказов в таблице user_actions.

Новую колонку с числом заказов назовите orders_count.

Результат отсортируйте по числу заказов по возрастанию.

Поля в результирующей таблице: action, orders_count

SELECT action,\
       count(action) as orders_count\
FROM   user_actions\
GROUP BY action\
ORDER BY orders_count


**############################################################**

Функция DATE_TRUNC используется для усечения дат и времени, т.е. она работает аналогично округлению ROUND, только для типов данных TIMESTAMP и INTERVAL.

SELECT DATE_TRUNC(part, column)

На месте part в кавычках указывается, до какой точности следует округлять переданное значение времени:  'year', 'month', 'day', 'hour' и т.д.

**############################################################**

**#3**

Задание:\
Используя группировку и функцию DATE_TRUNC, приведите все даты к началу месяца и посчитайте, сколько заказов было сделано в каждом из них.

Расчёты проведите по таблице orders. Колонку с усечённой датой назовите month, колонку с количеством заказов — orders_count.

Результат отсортируйте по месяцам — по возрастанию.

Поля в результирующей таблице: month, orders_count

SELECT date_trunc('month', creation_time) as month,\
       count(order_id) as orders_count\
FROM   orders\
GROUP BY month\
ORDER BY month


**#4**

Задание:\
Используя группировку и функцию DATE_TRUNC, приведите все даты к началу месяца и посчитайте, сколько заказов было сделано и сколько было отменено в каждом из них.

В этот раз расчёты проведите по таблице user_actions. Колонку с усечённой датой назовите month, колонку с количеством заказов — orders_count.

Результат отсортируйте сначала по месяцам — по возрастанию, затем по типу действия — тоже по возрастанию.

Поля в результирующей таблице: month, action, orders_count

SELECT action,\
       date_trunc('month', time) as month,\
       count(order_id) as orders_count\
FROM   user_actions\
GROUP BY month, action\
ORDER BY month, action


**#5**

По данным в таблице users посчитайте максимальный порядковый номер месяца среди всех порядковых номеров месяцев рождения пользователей сервиса. С помощью группировки проведите расчёты отдельно в двух группах — для пользователей мужского и женского пола.

Новую колонку с максимальным порядковым номером месяца рождения в группах назовите max_month. Преобразуйте значения в новой колонке в формат INTEGER, чтобы порядковый номер был выражен целым числом.

Результат отсортируйте по колонке с полом пользователей.

Поля в результирующей таблице: sex, max_month

SELECT sex,\
       max(date_part('month', birth_date))::integer as max_month\
FROM   users\
GROUP BY sex


**#6**

Задание:\
По данным в таблице users посчитайте порядковый номер месяца рождения самого молодого пользователя сервиса. С помощью группировки проведите расчёты отдельно в двух группах — для пользователей мужского и женского пола.

Новую колонку c порядковым номером месяца рождения самого молодого пользователя в группах назовите max_month. Преобразуйте значения в новой колонке в формат INTEGER, чтобы порядковый номер был выражен целым числом.

Результат отсортируйте по колонке с полом пользователей.

Поля в результирующей таблице: sex, max_month

SELECT sex,\
       date_part('month', max(birth_date))::integer as max_month\
FROM   users\
GROUP BY sex


**#7**

Задание:\
Посчитайте максимальный возраст пользователей мужского и женского пола в таблице users. Возраст измерьте числом полных лет.

Новую колонку с возрастом назовите max_age. Преобразуйте значения в новой колонке в формат INTEGER, чтобы возраст был выражен целым числом.

Результат отсортируйте по новой колонке по возрастанию возраста.

Поля в результирующей таблице: sex, max_age

SELECT sex,\
       max(date_part('year', age(current_date, birth_date))):: integer as max_age\
FROM   users\
GROUP BY sex\
ORDER BY max_age


**#8**

Задание:\
Разбейте пользователей из таблицы users на группы по возрасту (возраст по-прежнему измеряем числом полных лет) и посчитайте количество пользователей каждого возраста.

Колонку с возрастом назовите age, а колонку с числом пользователей — users_count. Преобразуйте значения в колонке с возрастом в формат INTEGER, чтобы возраст был выражен целым числом.

Результат отсортируйте по колонке с возрастом по возрастанию.

Поля в результирующей таблице: age, users_count

SELECT COUNT(user_id) AS  users_count,
       date_part('year', age(current_date, birth_date)) :: integer as age
FROM   users
GROUP BY age
ORDER BY age


**#9**

Задание:\
Вновь разбейте пользователей из таблицы users на группы по возрасту (возраст по-прежнему измеряем количеством полных лет), только теперь добавьте в группировку ещё и пол пользователя. Затем посчитайте количество пользователей в каждой половозрастной группе.

Все NULL значения в колонке birth_date заранее отфильтруйте с помощью WHERE.

Колонку с возрастом назовите age, а колонку с числом пользователей — users_count, имя колонки с полом оставьте без изменений. Преобразуйте значения в колонке с возрастом в формат INTEGER, чтобы возраст был выражен целым числом.

Отсортируйте полученную таблицу сначала по колонке с возрастом по возрастанию, затем по колонке с полом — тоже по возрастанию.

Поля в результирующей таблице: age, sex, users_count

SELECT date_part('year', age(birth_date)) :: integer as age,\
       sex,\
       count(user_id) as users_count\
FROM   users\
WHERE  birth_date is not null\
GROUP BY age, sex\
ORDER BY age, sex


**#10**

Задание:\
Посчитайте количество товаров в каждом заказе, примените к этим значениям группировку и рассчитайте количество заказов в каждой группе за неделю с 29 августа по 4 сентября 2022 года включительно. Для расчётов используйте данные из таблицы orders.

Выведите две колонки: размер заказа и число заказов такого размера за указанный период. Колонки назовите соответственно order_size и orders_count.

Результат отсортируйте по возрастанию размера заказа.

Поля в результирующей таблице: order_size, orders_count

SELECT\
  array_length(product_ids, 1) AS order_size,\
  COUNT(order_id) AS orders_count\
FROM\
  orders\
WHERE\
  creation_time :: text between '2022-08-29'\
  and '2022-09-05'\
GROUP BY\
  order_size\
ORDER BY\
  order_size
  

**#11**

Задание:\
Посчитайте количество товаров в каждом заказе, примените к этим значениям группировку и рассчитайте количество заказов в каждой группе. Учитывайте только заказы, оформленные по будням. В результат включите только те размеры заказов, общее число которых превышает 2000. Для расчётов используйте данные из таблицы orders.

Выведите две колонки: размер заказа и число заказов такого размера. Колонки назовите соответственно order_size и orders_count.

Результат отсортируйте по возрастанию размера заказа.

Поля в результирующей таблице: order_size, orders_count

Когда решите эту задачу, вернитесь к одной из предыдущих и подумайте, могли бы мы ещё каким-то способом сделать так, чтобы в результат не попала группа с NULL значениями. Можете самостоятельно написать ещё один запрос и попробовать сдать его в качестве альтернативного решения.

SELECT array_length(product_ids, 1) as order_size,\
       count(order_id) as orders_count\
FROM   orders\
WHERE  date_part('dow', creation_time) in (1, 2, 3, 4, 5)\
GROUP BY order_size having count(order_id) > 2000\
ORDER BY order_size


SELECT array_length(product_ids, 1) as order_size,\
       count(order_id) as orders_count\
FROM   orders\
WHERE  to_char(creation_time, 'Dy') not in ('Sat', 'Sun')\
GROUP BY order_size having count(order_id) > 2000\
ORDER BY order_size


**#12**

Задание:\
По данным из таблицы user_actions определите пять пользователей, сделавших в августе 2022 года наибольшее количество заказов.

Выведите две колонки — id пользователей и число оформленных ими заказов. Колонку с числом оформленных заказов назовите created_orders.

Результат отсортируйте сначала по убыванию числа заказов, сделанных пятью пользователями, затем по возрастанию id этих пользователей.

Поля в результирующей таблице: user_id, created_orders

SELECT user_id,\
       count(order_id) as created_orders\
FROM   user_actions\
WHERE  date_part('month', time) = 8\
   and action = 'create_order'\
GROUP BY user_id\
ORDER BY created_orders desc, user_id limit 5

Таким образом, мы снова можем дополнить список ключевых слов в запросе:

**SELECT**\
**FROM**\
**WHERE**\
**GROUP BY**\
**HAVING**\
**ORDER BY**\
**LIMIT**

**#13**

Задание:\
А теперь по данным таблицы courier_actions определите курьеров, которые в сентябре 2022 года доставили только по одному заказу.

В этот раз выведите всего одну колонку с id курьеров. Колонку с числом заказов в результат включать не нужно.

Результат отсортируйте по возрастанию id курьера.

Поле в результирующей таблице: courier_id

SELECT courier_id\
FROM   courier_actions\
WHERE  action = 'deliver_order'\
   and date_part('month', time) = 9\
   and date_part('year', time) = 2022\
GROUP BY courier_id having count(action) = 1\
ORDER BY courier_id


**#14**

Задание:\
Из таблицы user_actions отберите пользователей, у которых последний заказ был создан до 8 сентября 2022 года.

Выведите только их id, дату создания заказа выводить не нужно.

Результат отсортируйте по возрастанию id пользователя.

Поле в результирующей таблице: user_id

SELECT user_id\
FROM   user_actions\
WHERE  action = 'create_order'\
GROUP BY user_id having max(time) < '2022-09-08'\
ORDER BY user_id


**#15**

Задание:\
Разбейте заказы из таблицы orders на 3 группы в зависимости от количества товаров, попавших в заказ:

Малый (от 1 до 3 товаров);
Средний (от 4 до 6 товаров);
Большой (7 и более товаров).
Посчитайте число заказов, попавших в каждую группу. Группы назовите соответственно «Малый», «Средний», «Большой» (без кавычек).

Выведите наименования групп и число товаров в них. Колонку с наименованием групп назовите order_size, а колонку с числом заказов — orders_count.

Отсортируйте полученную таблицу по колонке с числом заказов по возрастанию.

Поля в результирующей таблице: order_size, orders_count

SELECT case when array_length(product_ids, 1) >= 1 and\
                 array_length(product_ids, 1) <= 3 then 'Малый'\
            when array_length(product_ids, 1) >= 4 and\
                 array_length(product_ids, 1) <= 6 then 'Средний'\
            else 'Большой' end as order_size,\
       count(order_id) as orders_count\
FROM   orders\
GROUP BY order_size\
ORDER BY count(order_id)


**############################################################**

Вданном случае данные выводились методом array_length() исходя из типа данных в конкретной колонке. В случае если нам будет необходимо подсчитать количество пользователей в возрастной группе, то для даты соответственно, нужно будет перевсети данные в тип integer.


**############################################################**


**#16**

Задание:\
Разбейте пользователей из таблицы users на 4 возрастные группы:

от 18 до 24 лет;\
от 25 до 29 лет;\
от 30 до 35 лет;\
старше 36.\
Посчитайте число пользователей, попавших в каждую возрастную группу. Группы назовите соответственно «18-24», «25-29», «30-35», «36+» (без кавычек).

В расчётах не учитывайте пользователей, у которых не указана дата рождения. Как и в прошлых задачах, в качестве возраста учитывайте число полных лет.

Выведите наименования групп и число пользователей в них. Колонку с наименованием групп назовите group_age, а колонку с числом пользователей — users_count.

Отсортируйте полученную таблицу по колонке с наименованием групп по возрастанию.\
Поля в результирующей таблице: group_age, users_count

SELECT case when date_part('year',\
                           age(current_date, birth_date)) :: integer >= 18 and\
                 date_part('year', age(current_date, birth_date)) :: integer <= 24 then '18-24'\
            when date_part('year', age(current_date, birth_date)) :: integer >= 25 and\
                 date_part('year', age(current_date, birth_date)) :: integer <= 29 then '25-29'\
            when date_part('year', age(current_date, birth_date)) :: integer >= 30 and\
                 date_part('year', age(current_date, birth_date)) :: integer <= 35 then '30-35'\
            else '36+' end as group_age,\
       count(user_id) as users_count\
FROM   users\
WHERE  birth_date is not null\
GROUP BY group_age\
ORDER BY group_age

##2

SELECT case when date_part('year', age(birth_date)) between 18 and\
                 24 then '18-24'\
            when date_part('year', age(birth_date)) between 25 and\
                 29 then '25-29'\
            when date_part('year', age(birth_date)) between 30 and\
                 35 then '30-35'\
            when date_part('year', age(birth_date)) >= 36 then '36+' end as group_age,\
       count(user_id) as users_count\
FROM   users\
WHERE  birth_date is not null\
GROUP BY group_age\
ORDER BY group_age


**#17**

Задание:\
По данным из таблицы orders рассчитайте средний размер заказа по выходным и будням.

Группу с выходными днями (суббота и воскресенье) назовите «weekend», а группу с будними днями (с понедельника по пятницу) — «weekdays» (без кавычек).

В результат включите две колонки: колонку с группами назовите week_part, а колонку со средним размером заказа — avg_order_size. 

Средний размер заказа округлите до двух знаков после запятой.

Результат отсортируйте по колонке со средним размером заказа — по возрастанию.

Поля в результирующей таблице: week_part, avg_order_size


SELECT
  CASE
    WHEN TO_CHAR(creation_time, 'Dy') not in ('Sat', 'Sun') THEN 'weekdays'
    ELSE 'weekend'
  END AS week_part,
  ROUND(AVG(array_length(product_ids, 1)), 2) AS avg_order_size
FROM
  orders
  GROUP BY week_part
  ORDER BY avg_order_size


**#13**